In [6]:
!pip install karateclub==1.2.0
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.2 MB/s eta 0:00:00
  Created wheel for karateclub: filename=karateclub-1.2.0-py3-none-any.whl size=94311 sha256=8466bf71b093512cb685e113dfbe826aa3ff39b818ca47a7fa702e55120bedb3
  Stored in directory: /root/.cache/pip/wheels/41/83/22/2ea49cf105f1a344e9f5813bc307112147af91b8480be84361
Successfully built karateclub
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.7 MB/s eta 0:00:00


In [7]:
!gdown --id "1RmrHId0d-uY7kJCSgCtNYbwYfp4Oum3c&export=download"
!unrar x -Y "/content/lab3.rar" -d "/content/"

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1RmrHId0d-uY7kJCSgCtNYbwYfp4Oum3c&export=download
To: /content/lab3.rar
100% 1.54M/1.54M [00:00<00:00, 73.7MB/s]

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/lab3.rar

Extracting  /content/lab3_attributes.csv                                   0%  OK 
Extracting  /content/facebook_features.json                               28%  OK 
Extracting  /content/facebook_target.csv                                  59%  OK 
Extracting  /content/lab3_edgelist.txt                                    59%  OK 
Extracting  /content/facebook_edges.csv                                  100%  OK 
All OK


In [8]:
# Task 1
import networkx as nx
from joblib import Parallel, delayed
import random
import itertools
import numpy as np
import pandas as pd

# Task 2
import json
import umap
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix
from karateclub.utils.walker import RandomWalker
from gensim.models.word2vec import Word2Vec
import seaborn as sns

In [9]:
def partition_num(num, workers):
    if num % workers == 0:
        return [num//workers]*workers
    else:
        return [num//workers]*workers + [num % workers]

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


def get_attributes_of_node(node_paths):
  node_paths_attributes = []
  # Get attribute (word) for each node
  df_attr = pd.read_csv("lab3_attributes.csv").astype(str)
  dict_attr = {}
  for i in range(len(df_attr)):
    dict_attr[df_attr.iloc[i, 0]] = df_attr.iloc[i, 1]
  for path in node_paths:
    for index, node in enumerate(path):
      path[index] = dict_attr[node]
    node_paths_attributes.append(path)
  return node_paths_attributes

def preprocessing(sentences):
    training_data = []
    for sentence in sentences:
        x = [word for word in sentence]
        training_data.append(x)
    return training_data


def prepare_data_for_training(sentences,w2v):
    data = {}
    for sentence in sentences:
        for word in sentence:
            if word not in data:
                data[word] = 1
            else:
                data[word] += 1
    V = len(data)
    data = sorted(list(data.keys()))
    vocab = {}
    for i in range(len(data)):
        vocab[data[i]] = i

    for sentence in sentences:
        for i in range(len(sentence)):
            center_word = [0 for x in range(V)]
            center_word[vocab[sentence[i]]] = 1
            context = [0 for x in range(V)]

            for j in range(i-w2v.window_size,i+w2v.window_size):
                if i!=j and j>=0 and j<len(sentence):
                    context[vocab[sentence[j]]] += 1
            w2v.X_train.append(center_word)
            w2v.y_train.append(context)
    w2v.initialize(V,data)

    return w2v.X_train,w2v.y_train

In [10]:
class word2vec():
    def __init__(self, window_size=2, n_hidden=10, learning_rate=0.001, epoch=300):
        self.window_size = window_size  # Size of the context window
        self.n_hidden = n_hidden        # Number of hidden neurons
        self.learning_rate = learning_rate  # Learning rate for SGD
        self.X_train = []               # Input vectors (one-hot encoded)
        self.y_train = []               # Target context vectors (one-hot encoded)
        self.V = None                   # Vocabulary size
        self.vocab = None               # Vocabulary list
        self.W1 = None                 # Weights between input and hidden layer
        self.W2 = None                  # Weights between hidden and output layer
        self.epoch = epoch
    def initialize(self, V, vocab):
        """
        Initialize the weights and other parameters.
        V: vocabulary size
        vocab: list of unique words
        """
        self.V = V
        self.vocab = vocab
        # Weight matrices initialization with small random numbers
        self.W1 = np.random.uniform(-1, 1, (self.V, self.n_hidden))  # V x n_hidden
        self.W2 = np.random.uniform(-1, 1, (self.n_hidden, self.V))  # n_hidden x V

    def forward(self, x):
        """
        Forward pass: computes the hidden layer output and the output layer (predicted context).
        x: input vector (one-hot encoded word)
        """
        h = np.dot(x, self.W1)  # h is the hidden layer output
        u = np.dot(h, self.W2)  # u is the output before applying softmax
        y_pred = self.softmax(u)  # y_pred is the predicted context distribution
        return h, y_pred

    def softmax(self, x):
        """Compute softmax values for each set of scores in x."""
        e_x = np.exp(x - np.max(x))  # numerical stability
        return e_x / np.sum(e_x)

    def backpropagate(self, x, h, y_pred, y_true):
        """
        Backpropagation: update weights based on prediction error.
        x: input vector (one-hot encoded word)
        h: hidden layer output
        y_pred: predicted context (softmax output)
        y_true: true context (one-hot encoded)
        """
        # # Error at output layer
        # error = y_pred - y_true

        # # Gradient with respect to W2
        # dW2 = np.outer(h, error)  # n_hidden x V

        # # Gradient with respect to W1
        # dh = np.dot(self.W2, error)  # Gradient from output to hidden
        # dW1 = np.outer(x, dh)  # V x n_hidden

        # # Update weights
        # self.W1 -= self.learning_rate * dW1
        # self.W2 -= self.learning_rate * dW2
        error = y_pred - y_true
        dW2 = np.outer(h, error)
        dh = np.dot(self.W2, error)
        dW1 = np.outer(x, dh)

        # Gradient clipping to prevent exploding gradients
        max_grad = 5.0
        dW1 = np.clip(dW1, -max_grad, max_grad)
        dW2 = np.clip(dW2, -max_grad, max_grad)

        self.W1 -= self.learning_rate * dW1
        self.W2 -= self.learning_rate * dW2
    def train(self):
        """
        Train the model using the training data (X_train and y_train).
        epochs: number of iterations over the dataset
        """
        for epoch in range(self.epoch):
            loss = 0
            for x, y_true in zip(self.X_train, self.y_train):
                # Forward pass
                h, y_pred = self.forward(np.array(x))

                # Compute loss (cross-entropy)
                loss += -np.sum(y_true * np.log(y_pred) )

                # Backpropagation
                self.backpropagate(np.array(x), h, y_pred, np.array(y_true))

            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")

    def get_word_vector(self, word):
        """
        Get the word vector for a given word from the W1 matrix.
        word: the word in the vocabulary
        """
        word_index = self.vocab.index(word)
        return self.W1[word_index]
    def predict(self, word, top_n=3):
        """
        Find the top N most similar words based on cosine similarity.
        word: the word in the vocabulary
        top_n: number of similar words to return
        """
        if word not in self.vocab:
            print(f"{word} not in vocabulary!")
            return None

        word_vector = self.get_word_vector(word)
        similarities = {}

        # Calculate similarity between the input word and all other words
        for other_word in self.vocab:
            if other_word != word:
                other_vector = self.get_word_vector(other_word)
                similarity = self.cosine_similarity(word_vector, other_vector)
                similarities[other_word] = similarity

        # Sort by similarity and return the top N most similar words
        sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
        return sorted_similarities[:top_n]
    def cosine_similarity(self, vec1, vec2):
        """Calculate cosine similarity between two vectors."""
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

Deepwalk

In [11]:
class RandomWalker:
  def __init__(self, G, num_walks, walk_length):
      """
      :param G: Graph
      :param num_walks: a number of walks
      :param walk_length: Length of a walk. Each walk is considered as a sentence
      """
      self.G = G
      self.num_walks = num_walks
      self.walk_length = walk_length


  def deepwalk_walk(self, start_node):
      """
      :param start_node: Starting node of a walk
      """
      walk = [start_node]
      while len(walk) < self.walk_length:
          cur = walk[-1]
          # Check if having any neighbors at the current node
          cur_nbrs = list(self.G.neighbors(cur))
          if len(cur_nbrs) > 0:
              # Random walk with the probability of 1/d(v^t). d(v^t) is the node degree
              walk.append(random.choice(cur_nbrs))
          else:
              break
      return walk


  def simulate_walks(self, workers=1, verbose=0):
      """
      :param workers: a number of workers running in parallel processing
      :param verbose: progress bar
      """
      G = self.G
      nodes = list(G.nodes())
      results = Parallel(n_jobs=workers, verbose=verbose)(
          delayed(self._simulate_walks)(nodes) for num in
          partition_num(self.num_walks, workers))
      walks = list(itertools.chain(*results))
      return walks


  # INFORMATION EXTRACTOR
  def _simulate_walks(self, nodes):
      walks = []
      # Iterate all walks per vertex
      for _ in range(self.num_walks):
          random.shuffle(nodes)
          # Iterate all nodes in a walk
          for v in nodes:
            walks.append(self.deepwalk_walk(start_node=v))
      return walks

In [12]:
class DeepWalk:
    def __init__(self, model, graph, walk_length, num_walks, workers=1):

        self.graph = graph
        self.w2v_model = model
        self._embeddings = {}

        self.walker = RandomWalker(graph, num_walks=num_walks, walk_length=walk_length)
        self.walks = self.walker.simulate_walks(workers=workers, verbose=1)
        self.sentences = get_attributes_of_node(self.walks)


    def train(self, window_size=5, epochs=100):
        print("Learning embedding vectors...")
        training_data = preprocessing(self.sentences)
        w2v = word2vec(window_size, epochs)
        prepare_data_for_training(training_data, w2v)
        w2v.train()
        print("Learning embedding vectors done!")
        self.w2v_model = w2v


    def test(self, word):
        print(self.w2v_model.predict(word,3))

Run graph

In [13]:
G = nx.read_edgelist('lab3_edgelist.txt',create_using=nx.DiGraph(),nodetype=None,data=[('weight',int)])# Read graph
model = DeepWalk(word2vec(), G, walk_length=3, num_walks=10, workers=1)
model.train(window_size=5)# train model

Learning embedding vectors...
Epoch 0, Loss: 8166.666899505466


<ipython-input-10-6b8e520fe445>:85: RuntimeWarning: divide by zero encountered in log
  loss += -np.sum(y_true * np.log(y_pred) )
<ipython-input-10-6b8e520fe445>:85: RuntimeWarning: invalid value encountered in multiply
  loss += -np.sum(y_true * np.log(y_pred) )


Epoch 100, Loss: nan
Epoch 200, Loss: nan
Learning embedding vectors done!


In [14]:
print(model.sentences)
model.test("to")
model.test("this")

[['something', 'you', 'you'], ['learned', 'I', 'to'], ['am', 'this', 'I'], ['this', 'happy', 'that'], ['in', 'all', 'this'], ['the', 'all', 'am'], ['to', 'happy', 'wish'], ['I', 'you', 'you'], ['you', 'this', 'I'], ['so', '!', 'am'], ['this', 'to', '!'], ['lab', 'happy', 'wish'], ['lab', '.', 'so'], ['happy', 'that', 'this'], ['.', 'so', 'I'], ['you', 'to', 'happy'], ['that', 'this', 'so'], ['all', 'so', '!'], ['I', 'this', 'to'], ['!', 'am', 'am'], ['wish', 'learned', 'I'], ['.', '.', 'so'], ['best', 'I', 'to'], ['.', 'so', 'that'], ['the', 'I', 'this'], ['you', '.', 'so'], ['in', 'all', 'to'], ['happy', 'wish', 'that'], ['something', 'happy', 'that'], ['am', 'this', 'I'], ['this', 'this', 'happy'], ['wish', 'all', 'this'], ['so', 'that', 'this'], ['I', 'to', 'lab'], ['that', 'you', 'you'], ['lab', 'happy', 'wish'], ['lab', '.', 'so'], ['learned', 'this', 'this'], ['this', 'so', 'lab'], ['you', 'you', 'you'], ['to', 'lab', 'happy'], ['I', 'to', 'happy'], ['best', 'to', 'lab'], ['all',

data

In [15]:
edges_path = 'facebook_edges.csv'
targets_path = 'facebook_target.csv'
features_path = 'facebook_features.json'

In [16]:
import pandas as pd
# Load edges
edges_df = pd.read_csv(edges_path)

# Load features
with open(features_path, 'r') as f:
    features = json.load(f)

# Create the graph
G = nx.from_pandas_edgelist(edges_df, 'id_1', 'id_2', create_using=nx.Graph())

nx.set_node_attributes(G, features, 'features')

# Initialize and train DeepWalk
# Define hyperparameters for DeepWalk
walk_length = 10
num_walks = 80
window_size = 5
workers = 2


In [17]:
# Train DeepWalk
model = DeepWalk(word2vec(), G, walk_length=walk_length, num_walks=num_walks, workers=workers)
model.train(window_size=window_size)


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  1.6min finished


KeyError: 14638